In [1]:
# Setup API
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("RB_Vehicle_1")


In [2]:
# Setup môi trường
# Cài đặt các gói, NHƯNG ép numpy phải ở phiên bản 1.x
!pip install "rfdetr[metrics]" roboflow "numpy<2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Check gpu
!nvidia-smi

Sun Oct 26 01:41:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
%%writefile main.py
# --- File main.py ---
# File này chứa TOÀN BỘ code: Tải data VÀ train
# ĐÃ SỬA LỖI RACE CONDITION

import os
import torch                     # <--- THÊM VÀO
import torch.distributed as dist # <--- THÊM VÀO
from kaggle_secrets import UserSecretsClient
from roboflow import Roboflow
from rfdetr import RFDETRBase  # <--- SỬA LỖI TYPO 'rfdert'

print("--- BẮT ĐẦU SCRIPT DDP TRÊN 2 GPU ---")

# --- PHẦN DDP SETUP MỚI ---
try:
    # Khởi tạo nhóm process, 'nccl' là backend cho GPU NVIDIA
    dist.init_process_group(backend='nccl')
    # Lấy ID của process này (sẽ là 0 hoặc 1)
    local_rank = int(os.environ['LOCAL_RANK'])
    # Đặt GPU cho process này
    torch.cuda.set_device(local_rank)
    print(f"Đã khởi tạo DDP cho Process Rank {local_rank} trên GPU {local_rank}.")
except Exception as e:
    print(f"Lỗi khi khởi tạo DDP (có thể đang chạy ở chế độ 1 GPU?): {e}")
    local_rank = 0 # Mặc định là 0 nếu không chạy DDP
# --- KẾT THÚC PHẦN DDP SETUP ---


# 1. Lấy API Key
# (Cả 2 process đều cần key để gọi Roboflow)
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("RB_Vehicle_1") 
rf = Roboflow(api_key=api_key) 

# Chỉ định đường dẫn dataset
# (ID project 3.3k ảnh của bạn: vehicle-detection-byizq-i3hsv)
DATASET_PATH = "/kaggle/working/Vehicle-detection-1" 

# 2. TẢI DATASET (CHỈ RANK 0 ĐƯỢC LÀM)
if local_rank == 0:
    print(f"Rank {local_rank} (Main): Bắt đầu tải dataset...")
    project = rf.workspace("myworkspace-jlyfn").project("vehicle-detection-byizq-i3hsv") 
    version = project.version(1) 
    # Kiểm tra xem data đã có chưa (phòng trường hợp restart)
    if not os.path.exists(DATASET_PATH):
        dataset = version.download("coco")
        print(f"Rank {local_rank} (Main): Đã tải và giải nén xong.")
    else:
        print(f"Rank {local_rank} (Main): Đã tìm thấy dataset, bỏ qua tải.")
else:
    print(f"Rank {local_rank} (Worker): Đang chờ Rank 0 tải dataset...")

# 3. BARRIER (QUAN TRỌNG)
# Tất cả process (0 và 1) phải đợi ở đây.
# Process 1 sẽ đợi cho đến khi Process 0 chạy xong phần code ở trên.
if dist.is_initialized():
    print(f"Rank {local_rank}: Đã đến barrier, đang đồng bộ...")
    dist.barrier()
print(f"Rank {local_rank}: Đã qua barrier. Tất cả data đã sẵn sàng.")

# 4. Cấu hình Huấn luyện
# (Cả 2 process đều chạy phần này)
OUTPUT_PATH = "/kaggle/working/RFDETR_Training_3.3k"
model = RFDETRBase()
print(f"Rank {local_rank}: Đã khởi tạo mô hình. Bắt đầu huấn luyện...")

# 5. Chạy Huấn luyện
# model.train() sẽ tự động phát hiện các biến môi trường DDP
model.train(
    dataset_dir = DATASET_PATH,
    output_dir = OUTPUT_PATH,
    epochs=300,
    batch_size = 4,      
    grad_accum_steps=4,
    lr=1e-4,
    tensorboard=True,
    early_stopping=True,
    early_stopping_patience=25, 
    early_stopping_min_delta=0.001 
)

print(f"--- HUẤN LUYỆN HOÀN TẤT (Process {local_rank}) ---")
print(f"Checkpoints được lưu tại: {OUTPUT_PATH}")

if dist.is_initialized():
    dist.destroy_process_group() # Dọn dẹp
# --- Kết thúc file main.py ---

Writing main.py


In [5]:
!python -m torch.distributed.launch --nproc_per_node=2 main.py

/usr/local/lib/python3.11/dist-packages/torch/distributed/launch.py:208: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use-env is set by default in torchrun.
If your script expects `--local-rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  main()
W1026 01:41:34.180000 82 torch/distributed/run.py:792] 
W1026 01:41:34.180000 82 torch/distributed/run.py:792] *****************************************
W1026 01:41:34.180000 82 torch/distributed/run.py:792] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1026 01:41:34.180000 82 torch/distributed/run.py:792] *****************************************
2025-10-26 01:41:49.59